<a href="https://colab.research.google.com/github/Pathin220/4105_ML_Hw5/blob/main/4105_Hw_5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import torch
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [106]:
#Repeat all sections of problem 2 using all the input features from the housing price dataset.

In [107]:

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Machine Learning/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
#mapping the variables to a binary output
varlist_1 =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist_1] = housing[varlist_1].apply(mapping)


In [109]:
#Splitting the Dataset
X_train, X_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)




In [110]:
#Splitting Variables
Y_train = X_train.pop("price")
Y_val = X_val.pop("price")

In [111]:
#Standardizing the Dataset
standard = StandardScaler()
vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
         'airconditioning', 'prefarea', 'furnishingstatus']

X_train_st = X_train
X_train_st[vars] = standard.fit_transform(X_train_st[vars])

X_val_st = X_val
X_val_st[vars] = standard.fit_transform(X_val_st[vars])

In [112]:
#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [113]:
#defining the model
def model_h(t_u, w12, w11, w10, w9, w8, w7, w6, w5, w4, w3, w2, w1, b):
  return w12*t_u**12 + w11*t_u**11 + w10*t_u**10 + w9*t_u**9 + w8*t_u**8 + w7*t_u**7 + w6*t_u**6 + w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b


In [114]:
#Setting up the initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [115]:
X_train_st_numpy = np.c_[np.ones((len(Y_train), 1)), X_train_st[vars]]
X_val_st_numpy = np.c_[np.ones((len(Y_val), 1)), X_val_st[vars]]

In [116]:
X_train_t = torch.tensor(X_train_st_numpy)
X_val_t = torch.tensor(X_val_st_numpy)
Y_train_t = torch.tensor(Y_train.values)
Y_val_t = torch.tensor(Y_val.values)

In [117]:
Y_train_m = Y_train_t.float().mean()
Y_train_st = Y_train.std()
Y_train_st_t = (Y_train_t-Y_train_m) / Y_train_st

In [118]:
#defining a training loop
def training_loop(n_epochs, optimizer, params, train_t_u,train_t_c, val_t_u, val_t_c):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:
       params.grad.zero_()

    t_p_train = model_h( train_t_u, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), train_t_c)

    t_p_val = model_h(val_t_u, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), val_t_c)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()


    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params



In [119]:
SGD_1_learning_rate = 1e-15
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD1 = optim.SGD([params], lr= SGD_1_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD1,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 131467027132112.6875, Validation loss: 35936793892416.8516

Epoch 3, Training loss: 67211620587508.2656, Validation loss: 28966503476866.1055

Epoch 500, Training loss: 26316996983323.4141, Validation loss: 25063440564244.7383

Epoch 1000, Training loss: 26316956615056.3164, Validation loss: 25063284860056.3477

Epoch 1500, Training loss: 26316916514058.9570, Validation loss: 25063130131141.7656

Epoch 2000, Training loss: 26316876500879.4570, Validation loss: 25062975467602.3086

Epoch 2500, Training loss: 26316836567309.3242, Validation loss: 25062820861225.9062

Epoch 3000, Training loss: 26316796809349.3477, Validation loss: 25062666846880.2461

Epoch 3500, Training loss: 26316757288790.0195, Validation loss: 25062513451609.3086

Epoch 4000, Training loss: 26316717961016.8789, Validation loss: 25062360467784.6680

Epoch 4500, Training loss: 26316678711317.9961, Validation los

tensor([-1.8183e-01,  7.9599e-01,  9.8157e-01,  9.9976e-01,  1.0029e+00,
         1.0008e+00,  1.0005e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  4.7538e-05], requires_grad=True)

In [120]:
SGD_2_learning_rate = 1e-20
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr= SGD_2_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296681410874585.5000, Validation loss: 54364553893646.4844

Epoch 3, Training loss: 296679378873929.6250, Validation loss: 54364325304914.2891

Epoch 500, Training loss: 295670419534583.8750, Validation loss: 54250826903156.2656

Epoch 1000, Training loss: 294657273187884.8750, Validation loss: 54136865349665.7969

Epoch 1500, Training loss: 293646035858708.1250, Validation loss: 54023126413310.9688

Epoch 2000, Training loss: 292636707547053.5000, Validation loss: 53909610094091.7422

Epoch 2500, Training loss: 291633698431503.0625, Validation loss: 53796811781043.7578

Epoch 3000, Training loss: 290641203121775.3125, Validation loss: 53685202512345.1094

Epoch 3500, Training loss: 289650567653569.6250, Validation loss: 53573810133151.1562

Epoch 4000, Training loss: 288661792026886.0625, Validation loss: 53462634643461.9141

Epoch 4500, Training loss: 287674876241724.5625, Vali

tensor([9.7811e-01, 9.9553e-01, 9.9911e-01, 9.9970e-01, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.7995e-10], requires_grad=True)

In [121]:
SGD_3_learning_rate = 1e-25
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD3 = optim.SGD([params], lr= SGD_3_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD3,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 1000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 1500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 4000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 4500, Training loss: 296683442882877.3750, Vali

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.7905e-15], requires_grad=True)

In [122]:
SGD_4_learning_rate = 1e-30
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD4 = optim.SGD([params], lr= SGD_4_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD4,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 1000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 1500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 3500, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 4000, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 4500, Training loss: 296683442882877.3750, Vali

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.7903e-20], requires_grad=True)

In [123]:
#Adam 1
Adam_1_learning_rate = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
Adam_1_optimizer = optim.Adam([params], lr= Adam_1_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = Adam_1_optimizer,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 244227149707464.5938, Validation loss: 48444982803921.2422

Epoch 3, Training loss: 197610155763582.6562, Validation loss: 43213275408720.7422

Epoch 500, Training loss: 26329257847778.0625, Validation loss: 25104353016655.0273

Epoch 1000, Training loss: 26320920173619.9219, Validation loss: 25087244782815.9961

Epoch 1500, Training loss: 26310415886434.2461, Validation loss: 25064677439609.2070

Epoch 2000, Training loss: 26298627027275.3945, Validation loss: 25038110880296.0664

Epoch 2500, Training loss: 26286369032988.8906, Validation loss: 25009113949460.4531

Epoch 3000, Training loss: 26274287907440.1406, Validation loss: 24979403926428.5859

Epoch 3500, Training loss: 26262670104236.4531, Validation loss: 24950644096395.8320

Epoch 4000, Training loss: 26251312309941.2891, Validation loss: 24924117732922.8203

Epoch 4500, Training loss: 26239576923999.7734, Validation lo

tensor([-2.1248e-01, -3.9326e-01,  4.6480e+00,  7.4529e+00,  2.9039e+01,
         4.0698e+01,  1.2172e+02,  1.6630e+02,  3.3453e+02,  3.8658e+02,
         4.8454e+02,  4.8196e+02,  5.0007e+02], requires_grad=True)

In [124]:
#Adam 2
Adam_2_learning_rate = 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
Adam_2_optimizer = optim.Adam([params], lr= Adam_2_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = Adam_2_optimizer,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 291183470521275.1875, Validation loss: 53742780790892.7109

Epoch 3, Training loss: 285741537154584.3125, Validation loss: 53127622183921.3906

Epoch 500, Training loss: 26333820848125.9219, Validation loss: 25113197710615.2148

Epoch 1000, Training loss: 26333434841000.7852, Validation loss: 25112483981028.8828

Epoch 1500, Training loss: 26332905483745.6758, Validation loss: 25111466231823.6680

Epoch 2000, Training loss: 26332216627445.3789, Validation loss: 25110105689196.1367

Epoch 2500, Training loss: 26331340378508.3203, Validation loss: 25108337844203.5273

Epoch 3000, Training loss: 26330239881199.4141, Validation loss: 25106076398995.3828

Epoch 3500, Training loss: 26328870133333.1445, Validation loss: 25103213208418.4219

Epoch 4000, Training loss: 26327178631870.0117, Validation loss: 25099617239345.8867

Epoch 4500, Training loss: 26325106538757.4141, Validation lo

tensor([-2.2231e-02,  1.1091e-01,  4.3998e-01,  7.0769e-01,  1.6644e+00,
         2.3079e+00,  5.7309e+00,  8.1246e+00,  2.1031e+01,  2.7578e+01,
         4.6649e+01,  4.6112e+01,  5.0324e+01], requires_grad=True)

In [125]:
#Adam 3
Adam_3_learning_rate = 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
Adam_3_optimizer = optim.Adam([params], lr= Adam_3_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = Adam_3_optimizer,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296130908741499.8750, Validation loss: 54302282834905.4609

Epoch 3, Training loss: 295578939806347.1250, Validation loss: 54239849901349.1484

Epoch 500, Training loss: 108986262078127.4531, Validation loss: 33398848066326.6406

Epoch 1000, Training loss: 42963819259218.8281, Validation loss: 26416790971029.3516

Epoch 1500, Training loss: 28159627377137.2266, Validation loss: 25110438523992.1211

Epoch 2000, Training loss: 26422349845908.1055, Validation loss: 25075814375607.5938

Epoch 2500, Training loss: 26335436242132.4609, Validation loss: 25107342692787.2695

Epoch 3000, Training loss: 26333913732294.2109, Validation loss: 25112932847857.4336

Epoch 3500, Training loss: 26333861574808.0977, Validation loss: 25113247655761.3555

Epoch 4000, Training loss: 26333805538794.3945, Validation loss: 25113150922521.2852

Epoch 4500, Training loss: 26333736383223.6445, Validation l

tensor([0.0208, 0.0243, 0.0328, 0.0398, 0.0641, 0.0807, 0.1694, 0.2351, 0.7028,
        1.0857, 3.6526, 3.5206, 5.1243], requires_grad=True)

In [126]:
#Adam 4
Adam_4_learning_rate = 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
Adam_4_optimizer = optim.Adam([params], lr= Adam_4_learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = Adam_4_optimizer,
    params = params,
    train_t_u = X_train_t,
    train_t_c = Y_train_t,
    val_t_u = X_val_t,
    val_t_c = Y_val_t,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296628154150209.5000, Validation loss: 54358528398194.2578

Epoch 3, Training loss: 296572871071625.2500, Validation loss: 54352274980506.0391

Epoch 500, Training loss: 270096059157998.8750, Validation loss: 51360679930647.7656

Epoch 1000, Training loss: 245385502725323.7500, Validation loss: 48575371933863.6328

Epoch 1500, Training loss: 222444863725455.4062, Validation loss: 45996380037033.4375

Epoch 2000, Training loss: 201139615667660.4688, Validation loss: 43608178503270.8750

Epoch 2500, Training loss: 181360422325986.1562, Validation loss: 41398148588049.9219

Epoch 3000, Training loss: 163019511405303.6875, Validation loss: 39356150155466.7344

Epoch 3500, Training loss: 146047148614983.2188, Validation loss: 37474105412357.3672

Epoch 4000, Training loss: 130387675092790.4688, Validation loss: 35745536471723.3672

Epoch 4500, Training loss: 115995969856810.4531, Vali

tensor([0.5421, 0.5421, 0.5422, 0.5422, 0.5422, 0.5423, 0.5424, 0.5426, 0.5436,
        0.5445, 0.5579, 0.5564, 0.5075], requires_grad=True)